* DSC530: Week 8
* 8.2 Exercise
* Marty Hoehler
* 5-5-24

# Exercise 9-1

First, we'll download data and import libraries as in weeks prior.

In [1]:
import numpy as np

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/scatter.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

import nsfg
import thinkstats2
import thinkplot
import first

live, firsts, others = first.MakeFrames()

In the text, they did two "difference in Means" tests, (one for the length of pregnancy, and one for the birth weight.)  They also did a correlation test between mother age and total weight and a Chi-Squared test of pregnancy length.  First, we'll bring in the classes from the text.  

Each of these classes has a "TestStatistic" funcion that defines the statistic we will be finding a p-value on.
 - For the difference in means, we are taking the absolute value of the difference of the two means.
 - For the correlation calculation, the test statistic is the absolute value of the correlation coefficient.
 - For the Chi squared test, the statistic is the sum of the squared differences of observed minus expected, divided by expected.
 
 

In [2]:
class DiffMeansPermute(thinkstats2.HypothesisTest):

    def TestStatistic(self, data):
        group1, group2 = data
        test_stat = abs(group1.mean() - group2.mean())
        return test_stat

    def MakeModel(self):
        group1, group2 = self.data
        self.n, self.m = len(group1), len(group2)
        self.pool = np.hstack((group1, group2))

    def RunModel(self):
        np.random.shuffle(self.pool)
        data = self.pool[:self.n], self.pool[self.n:]
        return data
    
class CorrelationPermute(thinkstats2.HypothesisTest):

    def TestStatistic(self, data):
        xs, ys = data
        test_stat = abs(thinkstats2.Corr(xs, ys))
        return test_stat

    def RunModel(self):
        xs, ys = self.data
        xs = np.random.permutation(xs)
        return xs, ys

class PregLengthTest(thinkstats2.HypothesisTest):

    def MakeModel(self):
        firsts, others = self.data
        self.n = len(firsts)
        self.pool = np.hstack((firsts, others))

        pmf = thinkstats2.Pmf(self.pool)
        self.values = range(35, 44)
        self.expected_probs = np.array(pmf.Probs(self.values))

    def RunModel(self):
        np.random.shuffle(self.pool)
        data = self.pool[:self.n], self.pool[self.n:]
        return data
    
    def TestStatistic(self, data):
        firsts, others = data
        stat = self.ChiSquared(firsts) + self.ChiSquared(others)
        return stat

    def ChiSquared(self, lengths):
        hist = thinkstats2.Hist(lengths)
        observed = np.array(hist.Freqs(self.values))
        expected = self.expected_probs * len(lengths)
        stat = sum((observed - expected)**2 / expected)
        return stat

Next, using the text as a guide, we'll create a function that runs the four tests.

In [3]:
def p_test(live2):

    # Labeling the main data sets as 'live2', 'firsts2', 'others2' to help me 
    # keep it straight from the versions defined above.
    n = len(live2)
    firsts2 = live2[live2.birthord == 1]
    others2 = live2[live2.birthord != 1]
    
    # Difference of means for the length of the pregnancy (first child vs other children)
    
    data = firsts2.prglngth.values, others2.prglngth.values
    ht = DiffMeansPermute(data)
    pvalue1 = ht.PValue()
    
    # Difference of means for the weight of the child (first child vs other children)
    # Note - the solution code recommended dropping null values for this section, so I've added that.
    data2 = firsts2.totalwgt_lb.dropna().values, others2.totalwgt_lb.dropna().values
    ht2 = DiffMeansPermute(data2)
    pvalue2 = ht2.PValue()
    
    # Correlation test for the age of the mother vs the weight of the child
    cleaned = live2.dropna(subset=['agepreg', 'totalwgt_lb'])
    data3 = cleaned.agepreg.values, cleaned.totalwgt_lb.values
    ht3 = CorrelationPermute(data3)
    pvalue3 = ht3.PValue()
    
    # Chi Squared test for the pregnancy length (first child vs other children)
    data4 = firsts2.prglngth.values, others2.prglngth.values
    ht4 = PregLengthTest(data4)
    pvalue4 = ht4.PValue()
    
    # Using the recommended formatting from the solution text to give us two decimals.
    print('%d\t%0.2f\t%0.2f\t%0.2f\t%0.2f' % (n, pvalue1, pvalue2, pvalue3, pvalue4))
    

#### Full Sample tests
Next, we'll find the p-values with the full sample. 

In [4]:
p_test(live)


9148	0.15	0.00	0.00	0.00


The p-values found in this recreation of the text examples are similar to the results that came from the book.  

#### Tests with smaller samples
Next, we'll use the recommended "thinkstats2.SampleRows" function to create samples of the "live" set that are various sizes.  First, I'd like to see a sample of about 6000.

In [22]:
live6000 = thinkstats2.SampleRows(live, 6000)
p_test(live6000)

6000	0.48	0.00	0.00	0.00


The p-value for test 1 has already started to increase with 6000 in the sample.

In [6]:
live3000 = thinkstats2.SampleRows(live, 3000)
p_test(live3000)

3000	0.34	0.03	0.00	0.00


In [7]:
live1000 = thinkstats2.SampleRows(live, 1000)
p_test(live1000)

1000	0.67	0.00	0.48	0.25


In [8]:
live500 = thinkstats2.SampleRows(live, 500)
p_test(live500)

500	0.56	0.26	0.09	0.04


In [9]:
live100 = thinkstats2.SampleRows(live, 100)
p_test(live100)

100	0.87	0.09	0.02	0.48


#### Conclusion
The p-tests are fluctuating more and more as the sample size n decreases.  For the most part, the tests that were positive with a large sample size become negative as we decrease the sample size.  But some positive results still happen, showing us the danger of using a small sample size.

# Exercise 10-1

First, we will import the BRFSS code and data and import the library.  Then, we'll build the dataset.

In [10]:


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/brfss.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/CDBRFS08.ASC.gz")
import brfss
import random

Next, we'll build the data frame for the heights and the log of the weights.

In [11]:
df = brfss.ReadBrfss(nrows=None)
df = df.dropna(subset=['htm3', 'wtkg2'])
heights, weights = df.htm3, df.wtkg2
log_weights = np.log10(weights)

"thinkstats2" provides a least squares formula for us:

In [12]:
inter, slope = thinkstats2.LeastSquares(heights, log_weights)
inter, slope

(0.993080416391762, 0.005281454169418105)

These parameters would not make very much intuitive sense in presentation.

### Explaining the Intercept
- The intercept would be the estimated log of the weight of someone that is 0 cm tall.  The book suggests that you instead present the intercept at the mean of x.  It's much more helpful to say "the expected weight of someone x cm tall is y" as opposed to "if there were a person 0 cm tall, we would expect the log of his weight to be .993080416391762."
- Instead, we'll find the mean height of the population and use our linear parameters to estimate the expected weight for that height.

In [13]:
# First, we find the log(weight) expected at the mean height using the linear formula y=mx+b
exp_log_wgt = slope*np.mean(heights)+ inter
# Now, we undo the base 10 log to find the expected weight.
exp_wgt = 10**exp_log_wgt
exp_wgt

76.80947249428195

From this, we can say that the a person of the mean height would be 76.8 kg.  This is much more intuitive than trying to explain the y intercept.

### Explaining the Slope

For a more intuitive explanation of the slope, I found this discussion:
http://www-stat.wharton.upenn.edu/~stine/stat621/handouts/LogsInRegression.pdf

The author points out:
- When you describe a linear slope, you describe it in terms of units of change.  
- When you describe a log-transformed linear scope, you can describe it in terms of percent change.

For our purposes, the linear model is:
    Log<sub>10</sub>(Weight) = slope * Height + inter
    
So we could use that calculation to determine what a 20% increase in weight would mean.
Log<sub>10</sub>(1.2 * Weight) = Log<sub>10</sub>(Weight) + Log<sub>10</sub>(1.2)

In [14]:
np.log10(1.2)

0.07918124604762482

So we could say that an increase in height of 8cm would increase the expected weight by 20%.

### Goodness of Fit
There were two methods mentioned in the text for judging goodness of fit.  

#### Comparing Residual error to Standard Deviation
First, is finding the RMSE of the residuals (called "Std(res)" in the text) and comparing it to the standard deviation of the data ("Std(ys)").

In [15]:
std_y = thinkstats2.Std(log_weights)
res = thinkstats2.Residuals(heights, log_weights, inter, slope)
std_res = thinkstats2.Std(res)

# Since weight is log-transformed, we'll convert them back to kg.
print('std_y:', 10**std_y, 'kg')
print('std_residuals:', 10**std_res, 'kg')

std_y: 1.2682569482092687 kg
std_residuals: 1.2229473795518997 kg


Based on this, the linear model improves the prediction of weight by about 46 grams.  

#### Coefficient of Determination:  R<sup>2</sup>
Another method of finding goodness of fit is to find R.  

In [16]:
R_2 = thinkstats2.CoefDetermination(log_weights, res)
R_2

0.2827349431189432

This can be interpreted as meaning that knowing the height of a person predicts about 28% of the variance in the log of the weight.

### Using Resampling to correct for weighted samples

We will use the thinkstats2 library's functions: "ResampleRows" and "ResampleRowsWeighted." 

The author also provided the following summary function (pg 121), using the thinkstats2 library of summary statistics:

In [17]:
def Summarize(estimates, actual=None):
    mean = thinkstats2.Mean(estimates)
    stderr = thinkstats2.Std(estimates, mu=actual)
    cdf = thinkstats2.Cdf(estimates)
    ci = cdf.ConfidenceInterval(90)
    print('mean, SE, CI', mean, stderr, ci)

In [18]:
estimates_unweighted = [thinkstats2.ResampleRows(df).htm3.mean() for _ in range(100)]
Summarize(estimates_unweighted)

mean, SE, CI 168.9570137078356 0.01356298568748001 (168.9338810404414, 168.9773312920633)


The "ResampleRowsWeighted()" function in thinkstats2 takes the weight field provided and uses it to assign a probability "p" which is used in the random choice calculation to create a sample that is re-weighted.  I used the "head()" function and confirmed that the weight column is called 'finalwt', and not 'totalwt' as it says in some copies of the text.

In [20]:
df.head()

,age,sex,wtyrago,finalwt,wtkg2,htm3
0,82.0,2,76.363636,185.870345,70.91,157.0
1,65.0,2,72.727273,126.603027,72.73,163.0
3,61.0,1,73.636364,517.926275,73.64,170.0
4,26.0,1,88.636364,1252.624630,88.64,185.0
5,42.0,1,118.181818,415.161314,109.09,183.0


In [21]:
estimates_weighted = [thinkstats2.ResampleRowsWeighted(df, 'finalwt').htm3.mean() for _ in range(100)]
Summarize(estimates_weighted)

mean, SE, CI 170.49923780795893 0.01719119267199965 (170.47426686068837, 170.5276809353463)


We can see that by resampling with the weights taken into account, we get a mean height that is about 1.5 cm taller.  
- The sampling error is only about .02 cm, so the difference in mean is much larger than what we would expect to see from re-sampling.  
- The confidence interval does not appear to be much changed in size.  In both the unweighted and the weighted, the spread is about .05cm.